Model import

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen2.5-Coder-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

prompt = "write a quick sort algorithm."
messages = [
    {"role": "system", "content": "You are Qwen, a highly skilled competitive python programmer. When given a prompt, you only output code. Your code should take in inputs using the input() function and return outputs using print."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 20.07it/s]
Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [4]:
print(response)

Certainly! Below is a Python implementation of the Quick Sort algorithm:

```python
def quick_sort(arr):
    if len(arr) <= 1:
        return arr
    else:
        pivot = arr[len(arr) // 2]
        left = [x for x in arr if x < pivot]
        middle = [x for x in arr if x == pivot]
        right = [x for x in arr if x > pivot]
        return quick_sort(left) + middle + quick_sort(right)

# Example usage:
if __name__ == "__main__":
    input_array = input("Enter a list of numbers separated by spaces: ")
    numbers = list(map(int, input_array.split()))
    sorted_array = quick_sort(numbers)
    print("Sorted array:", sorted_array)
```

This code defines a `quick_sort` function that sorts an array using the Quick Sort algorithm. It recursively partitions the array around a pivot element and sorts the sub-arrays. The example usage at the bottom demonstrates how to use the function by taking user input and printing the sorted array.


In [5]:

prompt = \
"""Write a program that reads an integer n from the input. The program should add five to n
, then triple the value and finally subtract ten. You are welcome to simplify the formula if you can, 
as long as the program gives the correct results.

Input:
Input consists of one line containing one integer n, where -1000 <= n <= 1000

Output
Output consists of one line containing one integer, the value computed from n as described above."""

messages = [
    {"role": "system", "content": "You are Qwen, a highly skilled competitive python programmer. When given a prompt, you only output code. Your code should take in inputs using the input() function and return outputs using print."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [ ]:
def generate_solution(description):
    prompt = description

    messages = [
        {"role": "system", "content": "You are Qwen, a highly skilled competitive python programmer. When given a prompt, you only output code. Your code should take in inputs using the input() function and return outputs using print."},
        {"role": "user", "content": prompt}
    ]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return response[9:-3]

In [9]:
response

'```python\nn = int(input())\nresult = (n + 5) * 3 - 10\nprint(result)\n```'

In [11]:
print(response[9:-3])


n = int(input())
result = (n + 5) * 3 - 10
print(result)



### part 1

In [14]:
import os
from dotenv import load_dotenv
from autokattis import OpenKattis
load_dotenv()
user = os.getenv("USER")
password = os.getenv("PASSWORD")
kt = OpenKattis(user, password)

[login] Candidate username(s): ['hugo-morvan']
[login] Successfully logged in to Kattis as hugo-morvan!
[database] Listed all available languages!
[database] Listed all 208 available countries!
[database] Listed all 3940 available universities!


In [ ]:
DIFFICULTY = 0 # 0-1 trivial, 2-3 easy, 4-5 moderate, 6-7 hard
PROG = '.py' # change to desired programming language file extension
INCLUDE_PB_LINK_IN_FILE = False
INCLUDE_PB_LINK_IN_CONSOLE = True
INCLUDE_SUBMIT_COMMAND = False
SOURCE_LOCATION = "./source/" #change to desired source directory

def get_problem():
    sug = kt.suggest()

    print(f"\nSelecting problem {sug[DIFFICULTY]['name']} \n")

    if INCLUDE_PB_LINK_IN_CONSOLE:
        print("link: ",sug[DIFFICULTY]["link"],"\n")
    
    path = SOURCE_LOCATION + sug[DIFFICULTY]['name']
    file_name = sug[DIFFICULTY]['pid'] + PROG
    file_path = os.path.join(path, file_name) 

    clean_path = SOURCE_LOCATION + "'" + sug[DIFFICULTY]['name'] + "'"
    clean_file_path = os.path.join(clean_path, file_name) 

    if INCLUDE_SUBMIT_COMMAND:
        submit_command = "kattis " + clean_file_path + " -f\n"
        print("Submit problem using:\n")
        print(submit_command)

    if os.path.exists(path):
        print("A directory already exists for this problem !\n")
    else:
        os.makedirs(path)
        print("The new directory is created!\n")
        with open(file_path, 'w') as f: 
            f.write(f'#{sug[0]["link"]}')

    return sug[DIFFICULTY]['pid'],clean_file_path

In [19]:
def get_description(pid):
    data = kt.problem(problem_ids=pid)
    description = data[0]['text']
    return description

In [ ]:
print(get_description('sequences'))

### part 2

In [52]:
import argparse
import configparser
import os
import re
import requests
import sys
import time

from lxml.html import fragment_fromstring


_DEFAULT_CONFIG = './'
_LANGUAGE_GUESS = {
    '.4th': 'Forth',
    '.c': 'C',
    '.c++': 'C++',
    '.cc': 'C++',
    '.c#': 'C#',
    '.cpp': 'C++',
    '.cs': 'C#',
    '.cxx': 'C++',
    '.cbl': 'COBOL',
    '.cob': 'COBOL',
    '.cpy': 'COBOL',
    '.forth': 'Forth',
    '.frt': 'Forth',
    '.fs': 'F#',
    '.fth': 'Forth',
    '.go': 'Go',
    '.hs': 'Haskell',
    '.java': 'Java',
    '.js': 'JavaScript (Node.js)',
    '.ts': 'TypeScript',
    '.kt': 'Kotlin',
    '.lisp': 'Common Lisp',
    '.cl': 'Common Lisp',
    '.m': 'Objective-C',
    '.ml': 'OCaml',
    '.pas': 'Pascal',
    '.php': 'PHP',
    '.pl': 'Prolog',
    '.py': 'Python 3',
    '.pyc': 'Python 3',
    '.rb': 'Ruby',
    '.rkt': 'Racket',
    '.rs': 'Rust',
    '.scala': 'Scala',
    '.f90': 'Fortran',
    '.f': 'Fortran',
    '.for': 'Fortran',
    '.sh': 'Bash',
    '.apl': 'APL',
    '.ss': 'Gerbil',
    '.jl': 'Julia',
    '.vb': 'Visual Basic',
    '.dart': 'Dart',
    '.zig': 'Zig',
    '.swift': 'Swift',
    '.nim': 'Nim',
    '.lua': 'Lua',
    '.pm': 'Perl',
    '.sno': 'SNOBOL',
    '.odin': 'Odin',
    '.a68': 'Algol 68',
    '.cr': 'Crystal',
    '.sim': 'Simula 67',
    '.d': 'D',
    '.mod': 'Modula-2',
    '.st': 'Smalltalk',
    '.adb': 'Ada',
    '.ads': 'Ada',
    '.erl': 'Erlang',
    '.ex': 'Elixir',
}

_GUESS_MAINCLASS = { 'Elixir', 'Erlang', 'Java', 'Kotlin', 'Modula-2', 'Scala' }
_GUESS_MAINFILE = {
    'Ada', 'Algol 68', 'APL', 'Bash', 'Crystal', 'Dart', 'Forth', 'Gerbil', 'JavaScript (Node.js)',
    'JavaScript (SpiderMonkey)', 'Julia', 'Common Lisp', 'Lua', 'Nim', 'Octave', 'Pascal', 'Perl', 'PHP',
	'Python 2', 'Python 3', 'Racket', 'Ruby', 'Rust', 'Simula', 'Smalltalk', 'SNOBOL', 'TypeScript', 'Zig',
}

_HEADERS = { 'User-Agent': 'kattis-cli-submit' }

_RUNNING_STATUS = 5
_COMPILE_ERROR_STATUS = 8
_ACCEPTED_STATUS = 16
_STATUS_MAP = {
    0: 'New', # <invalid value>
    1: 'New',
    2: 'Waiting for compile',
    3: 'Compiling',
    4: 'Waiting for run',
    _RUNNING_STATUS: 'Running',
    6: 'Judge Error',
    # 7: '<invalid value>',
    _COMPILE_ERROR_STATUS: 'Compile Error',
    9: 'Run Time Error',
    10: 'Memory Limit Exceeded',
    11: 'Output Limit Exceeded',
    12: 'Time Limit Exceeded',
    13: 'Illegal Function',
    14: 'Wrong Answer',
    # 15: '<invalid value>',
    _ACCEPTED_STATUS: 'Accepted',
}

class ConfigError(Exception):
    pass

def get_url(cfg, option, default):
    if cfg.has_option('kattis', option):
        return cfg.get('kattis', option)
    else:
        hostname = cfg.get('kattis', 'hostname')
        return f'https://{hostname}/{default}'

def guess_language(ext, files):
    if ext == ".C":
        return "C++"
    ext = ext.lower()
    if ext == ".h":
        if any(f.endswith(".c") for f in files):
            return "C"
        else:
            return "C++"
    if ext == ".py":
        return "Python 3"
    return _LANGUAGE_GUESS.get(ext, None)

def guess_mainfile(language, files):
    for filename in files:
        if os.path.splitext(os.path.basename(filename))[0] in ['main', 'Main']:
            return filename
    for filename in files:
        try:
            with open(filename) as f:
                conts = f.read()
                if language in ['Java', 'Rust', 'Scala', 'Kotlin'] and re.search(r' main\s*\(', conts):
                    return filename
                if language == 'Pascal' and re.match(r'^\s*[Pp]rogram\b', conts):
                    return filename
        except IOError:
            pass
    return files[0]

def guess_mainclass(language, files):
    if language in _GUESS_MAINFILE and len(files) > 1:
        return os.path.basename(guess_mainfile(language, files))
    if language in _GUESS_MAINCLASS:
        mainfile = os.path.basename(guess_mainfile(language, files))
        name = os.path.splitext(mainfile)[0]
        if language == 'Kotlin':
            return name[0].upper() + name[1:] + 'Kt'
        return name

def login(login_url, username, password=None, token=None):
    """Log in to Kattis.

    At least one of password or token needs to be provided.

    Returns a requests.Response with cookies needed to be able to submit
    """
    login_args = {'user': username, 'script': 'true'}
    if password:
        login_args['password'] = password
    if token:
        login_args['token'] = token

    return requests.post(login_url, data=login_args, headers=_HEADERS)

def login_from_config(cfg):
    """Log in to Kattis using the access information in a kattisrc file

    Returns a requests.Response with cookies needed to be able to submit
    """
    username = cfg.get('user', 'username')
    password = token = None
    try:
        password = cfg.get('user', 'password')
    except configparser.NoOptionError:
        pass
    try:
        token = cfg.get('user', 'token')
    except configparser.NoOptionError:
        pass
    if password is None and token is None:
        raise ConfigError('''\
Your .kattisrc file appears corrupted. It must provide a token (or a
KATTIS password).

Please download a new .kattisrc file''')

    loginurl = get_url(cfg, 'loginurl', 'login')
    return login(loginurl, username, password, token)

def submit(submit_url, cookies, problem, language, files, mainclass='', tag='', assignment=None, contest=None):
    """Make a submission.

    The url_opener argument is an OpenerDirector object to use (as
    returned by the login() function)

    Returns the requests.Result from the submission
    """

    data = {'submit': 'true',
            'submit_ctr': 2,
            'language': language,
            'mainclass': mainclass,
            'problem': problem,
            'tag': tag,
            'script': 'true'}

    if assignment is not None:
        data['assignment'] = assignment
    if contest is not None:
        data['contest'] = contest
    sub_files = [files]
    for f in files:
        with open(f) as sub_file:
            sub_files.append(('sub_file[]',
                              (os.path.basename(f),
                               sub_file.read(),
                               'application/octet-stream')))

    return requests.post(submit_url, data=data, files=sub_files, cookies=cookies, headers=_HEADERS)

def get_submission_url(submit_response, cfg):
    m = re.search(r'Submission ID: (\d+)', submit_response)
    if m:
        submissions_url = get_url(cfg, 'submissionsurl', 'submissions')
        submission_id = m.group(1)
        return f'{submissions_url}/{submission_id}'

def get_submission_status(submission_url, cookies):
    reply = requests.get(submission_url + '?json', cookies=cookies, headers=_HEADERS)
    return reply.json()

_RED_COLOR = 31
_GREEN_COLOR = 32
def color(s, c):
    return f'\x1b[{c}m{s}\x1b[0m'

def show_judgement(submission_url, cfg):
    print()
    login_reply = login_from_config(cfg)
    while True:
        status = get_submission_status(submission_url, login_reply.cookies)
        status_id = status['status_id']
        testcases_done = status['testcase_index']
        testcases_total = status['row_html'].count('<i') - 1

        status_text = _STATUS_MAP.get(status_id, f'Unknown status {status_id}')


        if status_id < _RUNNING_STATUS:
            print(f'\r{status_text}...', end='')
        else:
            print('\rTest cases: ', end='')

        if status_id == _COMPILE_ERROR_STATUS:
            print(f'\r{color(status_text, _RED_COLOR)}', end='')
            try:
                root = fragment_fromstring(status['feedback_html'], create_parent=True)
                error = root.find('.//pre').text
                print(color(':', _RED_COLOR))
                print(error, end='')
            except:
                pass
        elif status_id < _RUNNING_STATUS:
            print(f'\r{status_text}...', end='')
        else:
            print('\rTest cases: ', end='')

            if testcases_total == 0:
                print('???', end='')
            else:
                s = '.' * (testcases_done - 1)
                if status_id == _RUNNING_STATUS:
                    s += '?'
                elif status_id == _ACCEPTED_STATUS:
                    s += '.'
                else:
                    s += 'x'

                print(f'[{s: <{testcases_total}}]  {testcases_done} / {testcases_total}', end='')

        sys.stdout.flush()

        if status_id > _RUNNING_STATUS:
            # Done
            print()
            success = status_id == _ACCEPTED_STATUS
            try:
                root = fragment_fromstring(status['row_html'], create_parent=True)
                cpu_time = root.find('.//*[@data-type="cpu"]').text
                status_text += " (" + cpu_time + ")"
            except:
                pass
            if status_id != _COMPILE_ERROR_STATUS:
                print(color(status_text, _GREEN_COLOR if success else _RED_COLOR))
            return success

        time.sleep(0.25)

def get_config():
    """Returns a ConfigParser object for the .kattisrc file(s)
    """
    cfg = configparser.ConfigParser()

    if os.path.exists(_DEFAULT_CONFIG):
        cfg.read(_DEFAULT_CONFIG)

    try:
        file = __file__
    except NameError:
        file = sys.argv[0]

    if not cfg.read([os.path.join(os.path.expanduser("~"), '.kattisrc'),
                     os.path.join(os.path.dirname(file), '.kattisrc'),
                     os.path.join(os.path.dirname(os.path.realpath(file)), '.kattisrc')]):
        raise ConfigError('''\
I failed to read in a config file from your home directory or from the
same directory as this script. To download a .kattisrc file please visit
https://<kattis>/download/kattisrc

The file should look something like this:
[user]
username: yourusername
token: *********

[kattis]
hostname: <kattis>
loginurl: https://<kattis>/login
submissionurl: https://<kattis>/submit
submissionsurl: https://<kattis>/submissions''')
    return cfg

def submit_problem(files, problem):
    
    try:
        cfg = get_config()
    except ConfigError as exc:
        print(exc)
        sys.exit(1)
    
    problem, ext = problem, '.py'
    language = guess_language(ext, files)
    mainclass = guess_mainclass(language, files)
    tag = '-f'
    
    try:
        login_reply = login_from_config(cfg)
    except ConfigError as exc:
        print(exc)
        sys.exit(1)
    except requests.exceptions.RequestException as err:
        print('Login connection failed:', err)
        sys.exit(1)

    if not login_reply.status_code == 200:
        print('Login failed.')
        if login_reply.status_code == 403:
            print('Incorrect username or password/token (403)')
        elif login_reply.status_code == 404:
            print('Incorrect login URL (404)')
        else:
            print('Status code:', login_reply.status_code)
        sys.exit(1)

    submit_url = get_url(cfg, 'submissionurl', 'submit')

    try:
        result = submit(submit_url,
                        login_reply.cookies,
                        problem,
                        language,
                        files,
                        mainclass,
                        tag)
    except requests.exceptions.RequestException as err:
        print('Submit connection failed:', err)
        sys.exit(1)

    if result.status_code != 200:
        print('Submission failed.')
        if result.status_code == 403:
            print('Access denied (403)')
        elif result.status_code == 404:
            print('Incorrect submit URL (404)')
        else:
            print('Status code:', result.status_code)
        sys.exit(1)

    plain_result = result.content.decode('utf-8').replace('<br />', '\n')
    print(plain_result)

    submission_url = None
    try:
        submission_url = get_submission_url(plain_result, cfg)
    except configparser.NoOptionError:
        pass

    if submission_url:
        print(submission_url)
        if not show_judgement(submission_url, cfg):
            sys.exit(1)
    return None

In [ ]:
path = r'C:\Users\hugom\Documents\Kattis\Kattis-Solutions\source\3D Printer\3dprinter.py'
submit_problem(path , '3dprinter')


### part 3

In [ ]:
def write_solution(file_path, solution):
    with open(file_path, 'w') as f: 
            f.write(solution)

In [ ]:
pid, file_path = get_problem()
description = get_description(pid)
solution = generate_solution(description)
write_solution(solution)